Importing necessrary libraries

In [1]:
import numpy as np
import random
import torch
from torch.autograd import Variable
import torch.nn as nn
from scipy.special import expit, logit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import time

Generating Data randomly with a nonlinear calculation method for target

In [2]:
n_samples = 2000
random_state = np.random.RandomState(13)
x1 = random_state.uniform(size=n_samples)
x2 = random_state.uniform(size=n_samples)
x3 = random_state.randint(0, 4, size=n_samples)
x4 = random_state.uniform(size=n_samples) 
X = np.c_[x1,x2,x3,x4]

p = expit(np.sin(3 * x1) - 4 * x2 + x3)
Y = random_state.binomial(1, p, size=n_samples)

Standardizing and generating Train-Test Split

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X_std, Y, test_size=0.3, random_state=0)

Model Results Dict

In [4]:
Model_Results = {}

Sklearn Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

LR_model = LogisticRegression()
t_start = time.time()
LR_model.fit(X_train, Y_train)
t_end = time.time()

LR_training_time = t_end - t_start

Y_LR_Pred = LR_model.predict(X_train)
LR_train_accuracy = accuracy_score(Y_train, Y_LR_Pred)

Y_LR_Pred_t = LR_model.predict(X_test)
LR_test_accuracy = accuracy_score(Y_test, Y_LR_Pred_t)

Model_Results["Logistic Regression"] = (LR_train_accuracy, LR_test_accuracy, LR_training_time)
print(Model_Results["Logistic Regression"])

(0.745, 0.72, 0.004001617431640625)


SVM

In [7]:
from sklearn.svm import SVC

SVM_model = SVC()
t_start = time.time()
SVM_model.fit(X_train, Y_train)
t_end = time.time()

SVM_training_time = t_end - t_start

Y_SVM_Pred = SVM_model.predict(X_train)
SVM_train_accuracy = accuracy_score(Y_train, Y_SVM_Pred)

Y_SVM_Pred_t = SVM_model.predict(X_test)
SVM_test_accuracy = accuracy_score(Y_test, Y_SVM_Pred_t)

Model_Results["SVM"] = (SVM_train_accuracy, SVM_test_accuracy, SVM_training_time)
print(Model_Results["SVM"])

(0.7628571428571429, 0.715, 0.058847665786743164)


LGBM